In [ ]:
MIN_CONF = 0.3
NMS_THRESH = 0.3
MIN_DISTANCE = 50
MAX_RES = 40
import numpy as np
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def detect_people(frame,net,ln,personIdx=0):

  """
  Grabing the dimension of the frame and initilizing the list of resut
  """
  (H,W) =frame.shape[:2]
  results = []

  """
  Constructing blob from input frames to pass to YOLO object detector and
  also providing boundary boxes along with the associated probabilities
  """
  blob = cv2.dnn.blobFromImage(frame,1/255.0,(416,416),swapRB = True ,crop = False)
  net.setInput(blob)
  layer_output = net.forward(ln)

  """
  Initilising list of detected boxes,centroids and confidences
  """
  boxes = []
  centroids = []
  confidences = []

  # loop over each layer output

  for output in layer_output:
    #loop over each dectecction
    for detection in output:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]

      """
      Filtering the detection by ensuring the detected object is a person
      and it meets the minimum confidence
      """
      if class_id == personIdx and confidence > MIN_CONF:

        """
        Scaling the bounding box into the size of the image
        """
        box = detection[0:4]*np.array([W,H,W,H])
        (centerX,centerY,width,height) = box.astype('int')

        """
        Using x,y coordinates to derive the top right corner and left corner
        of bonding box.
        """
        x = int(centerX - (width/2))
        y = int(centerY - (height/2))

        """
        Updating the list of bounding box coordinates, centroids, and  confidence
        """
        boxes.append([x,y,int(width),int(height)])
        centroids.append((centerX,centerY))
        confidences.append(float(confidence))

  """
  applying non_maxima suppression to supress weak,overlaping bounding boxes
  """
  idxs = cv2.dnn.NMSBoxes(boxes,confidences,MIN_CONF,NMS_THRESH)

  #Ensuring atleast one detection exists
  if len(idxs) > 0:
    for i in idxs.flatten():
      (x,y) = (boxes[i][0],boxes[i][1])
      (w,h) =(boxes[i][2],boxes[i][3])

      # update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
      r = (confidences[i], (x, y, x + w, y + h), centroids[i])
      results.append(r)
  return results # return result
  


    






      




In [ ]:
from google.colab.patches import cv2_imshow
from scipy.spatial import distance as dist
import imutils
import argparse
import os

ap = argparse.ArgumentParser()
ap.add_argument("-i","--input",type=str,default="",help="path to (optional) input video file")
ap.add_argument("-o","--output",type=str,default="",help="path to (optional) output video file")
ap.add_argument("-d","--display",type=int,default=1,help="Whether or not output frame shold be displayed")
args = vars(ap.parse_args(["--input",'/content/drive/MyDrive/Colab Notebooks/Social distancing/Pexels Videos 2670.mp4',
                           "--output",'/content/drive/MyDrive/Colab Notebooks/Social distancing/out_put_1.avi','--display','1']))

#Loading coco class
label_path = os.path.sep.join(['/content/drive/MyDrive/Colab Notebooks/Social distancing/coco.names'])
labels = open(label_path).read().strip().split('\n')

#deriving the paths to the yolo weights and model configuration
weight_path = os.path.sep.join(['/content/drive/MyDrive/Colab Notebooks/Social distancing/yolov3.weights'])
config_path = os.path.sep.join(['/content/drive/MyDrive/Colab Notebooks/Social distancing/yolov3.cfg'])

#loading YOLO object detector trained on COCO dataset

net = cv2.dnn.readNetFromDarknet(config_path,weight_path)

#determine only the output layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

# initilizing video for output stream
vs = cv2.VideoCapture(args['input'] if args['input'] else 0)
writer = None

# Loop over the frames from the video stream

while True:
  (grabbed, frame) = vs.read()
  if not grabbed:
    break
  
  # Resizeing and detecting only the people in it
  frame = imutils.resize(frame,width=700)
  results = detect_people(frame,net,ln,personIdx=labels.index('person'))

  # make sure atleast 2 people are required for detection
  violate = set()
  if len(results)>=2:
    centroids = np.array([r[2] for r in results])
    D = dist.cdist(centroids,centroids,metric='euclidean')

    # loop over upper triangular of the distance matrix
    for i in range(0,D.shape[1]):
      for j in range(i+1,D.shape[1]):
        if D[i,j] < MIN_DISTANCE:
          violate.add(i)
          violate.add(j)
  
  #loop over the results
  for (i,(prop,bbox,centroid)) in  enumerate(results):
    (startX,startY,endX,endY) = bbox
    (cX,cY)= centroid
    color = (0,255,0)

    if i in violate :
      color = (0,0,255)

    #elif len(results) > MAX_RES:
      #color = (0,0,255)
    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.circle(frame, (cX, cY), 5, color, 1)
  
  text = "Social distance violation: {},Count : {},Maximum allowed people: {}".format(len(violate),len(results),MAX_RES)
  cv2.putText(frame,text,(10,frame.shape[0]-38),cv2.FONT_HERSHEY_SIMPLEX,0.60,(0,0,255),2)

  if len(results) > MAX_RES:
    text = "Excess people: {}".format(len(results)-MAX_RES)
    cv2.putText(frame,text,(10,frame.shape[0]-15),cv2.FONT_HERSHEY_COMPLEX,0.75,(0,0,255),2)



  if args['display'] > 0:
    cv2_imshow(frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
      break

  if args['output'] != " " and writer is None:
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(args["output"], fourcc, 25,(frame.shape[1], frame.shape[0]), True)

  if writer is not None:
    writer.write(frame)

  






Output hidden; open in https://colab.research.google.com to view.